In [10]:
import numpy as np
from scipy.stats import entropy

class SparsePro:
    def __init__(self, P, K):
        self.p = P
        self.k = K
        self.gamma = np.zeros((self.p, self.k))

    def get_effect(self, cthres=0.95, ethres=20):
        vidx = np.argsort(-self.gamma, axis=1)
        matidx = np.argsort(-self.gamma, axis=0)
        mat_eff = np.zeros((self.p, self.k))  # effective gamma
        for p in range(self.p):
            mat_eff[p, vidx[p, 0]] = self.gamma[p, vidx[p, 0]]
        csum = mat_eff.sum(axis=0).round(2)
        print("Attainable coverage for effect groups: {}".format(csum))  # statistical evidence
        eff = {}
        eff_gamma = {}
        for k in range(self.k):
            if csum[k] > cthres:
                if entropy(mat_eff[:, k]) < np.log(ethres):
                    for p in range(self.p):
                        if np.sum(mat_eff[matidx[0:p, k], k]) > cthres * csum[k] or mat_eff[matidx[p, k], k] < 0.01:
                            eff[k] = matidx[0:p, k]
                            eff_gamma[k] = mat_eff[eff[k], k].round(4)
                            break
        return eff, eff_gamma

# Create an instance of SparsePro
model = SparsePro(P=5, K=3)

# Generate random gamma values
np.random.seed(42)  # Set seed for reproducibility
gamma_values = np.random.dirichlet(np.ones(model.k), size=model.p)
model.gamma = gamma_values

# Call the get_effect method
eff, eff_gamma = model.get_effect(cthres=0.95, ethres=20)

# Print the results
print("Gamma values:")
print(model.gamma)
print("Effective variants:")
print(eff)
print("Effective gamma values:")
print(eff_gamma)

Attainable coverage for effect groups: [1.53 1.3  0.74]
Gamma values:
[[0.09784297 0.62761396 0.27454307]
 [0.729092   0.13546541 0.13544259]
 [0.02001195 0.67261832 0.30736973]
 [0.25890493 0.00437364 0.73672143]
 [0.80260249 0.10723697 0.09016054]]
Effective variants:
{0: array([4, 1]), 1: array([2, 0])}
Effective gamma values:
{0: array([0.8026, 0.7291]), 1: array([0.6726, 0.6276])}
